# Хакатон AI.Hack Москва.
## Газпром Нефть

https://www.kaggle.com/c/ai-hack-2018-msk-gpn/leaderboard

In [1]:
import pandas as pd
import numpy as np
import gc

In [2]:
# предопределим типы колонок для экономии памяти
dtypes = {
    'ID': np.int32,
    'CLIENT': np.int32,
    'DATA_TRANS': str,
    'AZS_NUMBER': np.int16,
    'REGION_AZS': np.int16,
    'VID_NP': np.int16,
    'COL_LITR':  np.float32,
    'CENA_CLIENT': np.float32
    }

In [3]:
train = pd.read_csv('datasets/gpn_b2b/train_data.csv', dtype = dtypes)

In [4]:
test = pd.read_csv('datasets/gpn_b2b/test_data.csv', dtype = dtypes)

In [5]:
def month(s): #для обучения сезонности
    return int(s[3:5])

def year(s):
    return int(s[8:10])

def date(s): #для пристегивания фич
    return s[3:10]

In [6]:
%%time

train['month'] = train['DATA_TRANS'].apply(month).astype(float)

test['month'] = test['DATA_TRANS'].apply(month).astype(float)

CPU times: user 24.7 s, sys: 931 ms, total: 25.6 s
Wall time: 25.6 s


In [7]:
train_sel = train  # catboost не хватало оперативы и использовал только часть, удалено, т.к. lightgbm показал себя на этих данных лучше

In [9]:
%%time

features = pd.read_csv('datasets/gpn_b2b/features_monthly.csv') 
# сгенерировано с полным тестом блокнотом features_monthly.ipynb
# внезапно медианный литраж с разбивкой по месяцам + линейная регрессия на янв-мар 2018 
#дает хуже скор чем по всему периоду

trans_count_d = {} # немножко покешируем т.к. вытаскивание из датафрейма медленнее
ltr_mean_d = {}

for index, row in features.iterrows():
    trans_count_d[row['client_date']] = row['trans_count']
    ltr_mean_d[row['client_date']] = row['ltr_mean']

def trans_count(c):
    try:
        return trans_count_d[c]
    except:
        return np.nan

def ltr_mean(c):
    try:
        return ltr_mean_d[c]
    except:
        return np.nan

CPU times: user 48.9 s, sys: 73 ms, total: 49 s
Wall time: 49 s


In [10]:
%%time

train_sel['date'] = train_sel['DATA_TRANS'].apply(date)
train_sel['client_date'] = train_sel['CLIENT'].astype(str) + '_' + train_sel['date'].astype(str)
train_sel['trans_count'] = train_sel['client_date'].apply(trans_count).astype(float)
train_sel['ltr_mean'] = train_sel['client_date'].apply(ltr_mean).astype(float)

test['date'] = test['DATA_TRANS'].apply(date)
test['client_date'] = test['CLIENT'].astype(str) + '_' + test['date'].astype(str)
test['trans_count'] = test['client_date'].apply(trans_count).astype(float)
test['ltr_mean'] = test['client_date'].apply(ltr_mean).astype(float)

CPU times: user 1min 52s, sys: 6.23 s, total: 1min 58s
Wall time: 1min 58s


In [11]:
%%time

# фичи про оценку размера бизнеса
# до 40 000 литров/месяц – массовый;
# от 40 000 до 150 000 литров/месяц – средний;
# от 150 000 литров/месяц – крупный;

train_sel['biz_estimated_ltr'] = train_sel['trans_count'] * train_sel['ltr_mean']
train_sel['biz_mass'] = train_sel['biz_estimated_ltr'].apply(lambda x: 1 if x <= 40000 else 0).astype(np.int16)
train_sel['biz_mid'] = train_sel['biz_estimated_ltr'].apply(lambda x: 1 if x > 40000 and x <= 150000 else 0).astype(np.int16)
train_sel['biz_big'] = train_sel['biz_estimated_ltr'].apply(lambda x: 1 if x > 150000 else 0).astype(np.int16)


test['biz_estimated_ltr'] = test['trans_count'] * test['ltr_mean']
test['biz_mass'] = test['biz_estimated_ltr'].apply(lambda x: 1 if x <= 40000 else 0).astype(np.int16)
test['biz_mid'] = test['biz_estimated_ltr'].apply(lambda x: 1 if x > 40000 and x <= 150000 else 0).astype(np.int16)
test['biz_big'] = test['biz_estimated_ltr'].apply(lambda x: 1 if x > 150000 else 0).astype(np.int16)

CPU times: user 47.1 s, sys: 4.48 s, total: 51.6 s
Wall time: 1min 5s


In [12]:
train_sel.sample(5)

,ID,CLIENT,DATA_TRANS,AZS_NUMBER,REGION_AZS,VID_NP,COL_LITR,CENA_CLIENT,month,date,client_date,trans_count,ltr_mean,biz_estimated_ltr,biz_mass,biz_mid,biz_big
15226606,15226606,28300,08.07.2017 16:40:41,1686,4,1,400.000000,38.090000,7.0,07.2017,28300_07.2017,18904.0,178.742401,3.378946e+06,0,0,1
7865285,7865285,24630,21.03.2017 4:23:58,2518,5,1,81.669998,33.490002,3.0,03.2017,24630_03.2017,1353.0,246.990585,3.341783e+05,0,0,1
28962808,28962808,5124,29.11.2017 13:41:35,750,21,1,35.000000,35.779999,11.0,11.2017,5124_11.2017,6564.0,66.078438,4.337389e+05,0,0,1
6587793,6587793,34300,27.02.2017 7:03:43,2714,14,2,40.000000,33.299999,2.0,02.2017,34300_02.2017,46.0,38.182240,1.756383e+03,1,0,0
8883112,8883112,23303,06.04.2017 10:54:30,2484,3,2,30.000000,38.230000,4.0,04.2017,23303_04.2017,414.0,38.769535,1.605059e+04,1,0,0


In [13]:
test.sample(5)

,ID,CLIENT,DATA_TRANS,AZS_NUMBER,REGION_AZS,VID_NP,CENA_CLIENT,month,date,client_date,trans_count,ltr_mean,biz_estimated_ltr,biz_mass,biz_mid,biz_big
999009,999009,34917,29.01.2018 9:46:26,1328,17,1,39.669998,1.0,01.2018,34917_01.2018,301.0,122.249344,3.679705e+04,1,0,0
375183,375183,48661,24.01.2018 9:19:47,1547,5,1,36.770000,1.0,01.2018,48661_01.2018,362.0,299.312531,1.083511e+05,0,1,0
62614,62614,1,13.03.2018 21:54:42,1430,21,1,40.500000,3.0,03.2018,1_03.2018,80424.0,182.545700,1.468106e+07,0,0,1
269683,269683,23205,05.03.2018 17:13:00,1749,19,1,39.900002,3.0,03.2018,23205_03.2018,37.0,67.232681,2.487609e+03,1,0,0
881927,881927,33530,08.02.2018 13:29:35,1781,24,2,38.150002,2.0,02.2018,33530_02.2018,31.0,30.399511,9.423849e+02,1,0,0


In [14]:
cols = ['CLIENT','AZS_NUMBER','REGION_AZS','VID_NP','CENA_CLIENT','month','trans_count','biz_estimated_ltr','ltr_mean']

In [15]:
X_train = train_sel[cols]
y = train_sel['COL_LITR']
X_test = test[cols]

del train, train_sel, test
gc.collect() 

393

In [30]:
import lightgbm

In [31]:
%%time

ctb = lightgbm.LGBMRegressor(n_estimators=40)
ctb.fit(X_train, y)

CPU times: user 5min 4s, sys: 5.38 s, total: 5min 9s
Wall time: 1min 36s


In [32]:
ctb

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       learning_rate=0.1, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=40,
       n_jobs=-1, num_leaves=31, objective=None, random_state=None,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=1)

In [33]:
pd.DataFrame(X_train.columns, ctb.feature_importances_)

,0
43,CLIENT
292,AZS_NUMBER
270,REGION_AZS
87,VID_NP
89,CENA_CLIENT
4,month
60,trans_count
80,biz_estimated_ltr
275,ltr_mean


In [34]:
y_test = ctb.predict(X_test)

In [35]:
y_test.shape

(1000000,)

In [36]:
X_test.shape

(1000000, 9)

In [37]:
sample_submission = pd.read_csv('datasets/gpn_b2b/sample_submission.csv')

In [38]:
sample_submission.tail()

,ID,COL_LITR
999995,999995,0.0
999996,999996,0.0
999997,999997,0.0
999998,999998,0.0
999999,999999,0.0


In [39]:
sample_submission['COL_LITR'] = y_test

In [40]:
sample_submission.tail()

,ID,COL_LITR
999995,999995,152.634069
999996,999996,169.025747
999997,999997,336.789013
999998,999998,39.789945
999999,999999,36.891937


In [41]:
sample_submission.to_csv('gpn_b2b_allm.csv.gz', compression='gzip',  index=False)